# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770811214424                   -0.53    8.0         
  2   -2.772136817816       -2.88       -1.31    1.0    172ms
  3   -2.772169752467       -4.48       -2.59    1.0    198ms
  4   -2.772170713382       -6.02       -3.68    2.0    200ms
  5   -2.772170721036       -8.12       -3.90    2.0    215ms
  6   -2.772170722936       -8.72       -4.80    1.0    209ms
  7   -2.772170723013      -10.11       -5.41    2.0    208ms
  8   -2.772170723015      -11.89       -5.88    1.0    200ms
  9   -2.772170723015      -12.43       -6.71    2.0    227ms
 10   -2.772170723015   +  -14.10       -7.49    1.0    219ms
 11   -2.772170723015      -14.27       -8.93    2.0    219ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770779798526                   -0.53    9.0         
  2   -2.7

1.7735580275324858

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770783780360                   -0.52    8.0         
  2   -2.772058229842       -2.89       -1.32    1.0    175ms
  3   -2.772083139888       -4.60       -2.49    1.0    175ms
  4   -2.772083359686       -6.66       -3.24    1.0    254ms
  5   -2.772083415177       -7.26       -3.83    2.0    202ms
  6   -2.772083417764       -8.59       -5.41    1.0    227ms
  7   -2.772083417807      -10.37       -5.23    3.0    230ms
  8   -2.772083417811      -11.40       -6.47    1.0    200ms
  9   -2.772083417811      -13.62       -6.47    2.0    225ms
 10   -2.772083417811      -13.71       -7.25    1.0    203ms
 11   -2.772083417811   +  -14.40       -7.86    1.0    203ms
 12   -2.772083417811   +  -14.24       -8.96    2.0    246ms

Polarizability via ForwardDiff:       1.7725349701317987
Polarizability via finite difference: 1.7735580275324858
